# Final Project Spark Notebook

### Small Data

### Make all scripts executable

In [1]:
!!chmod a+x ./*/*.py

["chmod: changing permissions of './P1/spark_p1.py': Operation not permitted",
 "chmod: changing permissions of './P2/spark_p2.py': Operation not permitted",
 "chmod: changing permissions of './P3/spark_p3.py': Operation not permitted",
 "chmod: changing permissions of './P4/spark_p4.py': Operation not permitted",
 "chmod: changing permissions of './P5/spark_p5.py': Operation not permitted"]

### Remove all Results

In [2]:
!rm -rf ./*/*.result

### P1

In [ ]:
# %load ./P1/spark_p1.py
import pyspark
from operator import add as sum

sc = pyspark.SparkContext('local[*]')
try:
    lines = sc.textFile('../data/epa_hap_daily_summary-small.csv') # Change the name of the file to what you have it named here
    logTuples = lines.filter( lambda line: len(line) > 0) \
                     .zipWithIndex() \
                     .filter( lambda x: x[1] > 0) \
                     .map(lambda x: x[0]) \
                     .map( lambda line: line.split(',')) \
                     .map( lambda arr: ( (arr[0], arr[1], arr[2]), arr[24]) )                    
    DistinctTuples = logTuples.distinct()
    MonitorCounts = DistinctTuples.map( lambda x: (x[1], 1) ) \
                                  .reduceByKey(sum) \
                                  .sortBy( lambda x: x[1], ascending = False)
    rank = 1
    for state,number in MonitorCounts.collect():
        print(f"{rank}. {state}:{number}")
        rank += 1
    sc.stop()
except Exception as err:
    print(err)
    sc.stop()


In [22]:
!!time python ./P1/spark_p1.py > ./P1/p1.result

['21/12/22 16:58:23 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable',
 "Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties",
 'Setting default log level to "WARN".',
 'To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).',
 '',
 '[Stage 0:>                                                          (0 + 4) / 4]',
 '[Stage 0:==============>                                            (1 + 3) / 4]',
 '                                                                                ',
 '',
 '[Stage 1:>                                                          (0 + 4) / 4]',
 '[Stage 1:==============>                                            (1 + 3) / 4]',
 '                                                                                ',
 'real\t0m11.621s',
 'user\t0m0.834s',
 'sys\t0m0.925s']

In [23]:
!cat ./P1/p1.result

1. California:162
2. Texas:132
3. Minnesota:94
4. Ohio:89
5. Michigan:84
6. New York:66
7. South Carolina:64
8. Montana:60
9. Pennsylvania:60
10. Indiana:52
11. Colorado:51
12. Florida:50
13. Illinois:50
14. North Carolina:49
15. Washington:42
16. Louisiana:40
17. Arizona:38
18. Kansas:37
19. Georgia:34
20. Oregon:31
21. Kentucky:30
22. Alabama:28
23. Tennessee:27
24. New Jersey:24
25. Wisconsin:24
26. Vermont:22
27. Mississippi:21
28. Maine:21
29. Oklahoma:20
30. Virginia:20
31. Massachusetts:19
32. Iowa:18
33. Maryland:17
34. New Mexico:17
35. Idaho:17
36. Country Of Mexico:16
37. Missouri:15
38. Connecticut:15
39. New Hampshire:12
40. Rhode Island:12
41. Utah:12
42. Alaska:12
43. West Virginia:10
44. Arkansas:10
45. Wyoming:9
46. Nevada:8
47. North Dakota:7
48. South Dakota:7
49. Nebraska:6
50. Delaware:6
51. Puerto Rico:6
52. District Of Columbia:5
53. Hawaii:5
54. Virgin Islands:4


### P2

In [ ]:
# %load ./P2/spark_p2.py
import pyspark

sc = pyspark.SparkContext('local[*]')

try:
    lines = sc.textFile('../data/epa_hap_daily_summary-small.csv') # Change the name of the file to what you have it named here
    logTuples = lines.filter( lambda line: len(line) > 0) \
                     .zipWithIndex() \
                     .filter( lambda x: x[1] > 0) \
                     .map(lambda x: x[0]) \
                     .map( lambda line: line.split(','))
    
    counties_airqual = logTuples.map(lambda l: [(l[0]+l[1], l[25]), float(l[16])])
    airqual = counties_airqual.mapValues(lambda v: (v, 1))\
                    .reduceByKey(lambda a,b: (a[0]+b[0], a[1]+b[1]))\
                    .mapValues(lambda v: v[0]/v[1])


    ranking = airqual.sortBy(lambda l: l[1], ascending=False)
    
    rank = 1
    for (state, value) in ranking.collect():
        print(f"{rank}: {state[1]}, {value}")
        rank +=1

    sc.stop()
except Exception as e:
    print(e)
    sc.stop()

In [25]:
!!time python ./P2/spark_p2.py > ./P2/p2.result

['21/12/22 17:02:51 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable',
 "Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties",
 'Setting default log level to "WARN".',
 'To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).',
 '',
 '[Stage 0:>                                                          (0 + 4) / 4]',
 '[Stage 0:==============>                                            (1 + 3) / 4]',
 '                                                                                ',
 '',
 '[Stage 1:>                                                          (0 + 4) / 4]',
 '[Stage 1:==============>                                            (1 + 3) / 4]',
 '                                                                                ',
 'real\t0m9.401s',
 'user\t0m0.725s',
 'sys\t0m0.890s']

In [26]:
!cat ./P2/p2.result

1: Tipton, 2556.0
2: Nassau, 19.0
3: Columbiana, 7.385690735785953
4: Park, 5.611212121212121
5: CHIHUAHUA STATE, 4.5121875
6: Caldwell, 4.116666666666667
7: Kings, 3.9843770491803276
8: Madera, 3.7393
9: Franklin, 3.3499999999999996
10: Jefferson, 3.07
11: Oakland, 2.888877848101266
12: Lake, 2.879328647058823
13: Duval, 2.7794603978494625
14: Middlesex, 2.6500000000000004
15: Kearny, 2.3753333333333333
16: Bucks, 2.3674999999999997
17: San Luis Obispo, 2.3333333333333335
18: Edgecombe, 2.325
19: Pawnee, 2.2941176470588234
20: Westchester, 2.239375
21: Johnston, 2.225
22: Hartford, 2.0787055896226416
23: Granville, 2.0285714285714285
24: Asotin, 2.025
25: Duplin, 2.0
26: Boulder, 1.960470901234568
27: Crittenden, 1.9000000000000001
28: Yancey, 1.9
29: Los Angeles, 1.8391350859879247
30: Wayne, 1.8262007365148463
31: Iberville, 1.8213132266666665
32: Caswell, 1.80075
33: Chatham, 1.8
34: Pitt, 1.7999999999999998
35: Clinton, 1.7583018867924527
36: Imperial, 1.6478600515463917
37: Jeffe

### P3

In [ ]:
# %load ./P3/spark_p3.py
import pyspark

sc = pyspark.SparkContext('local[*]')
try:
    lines = sc.textFile('../data/epa_hap_daily_summary-small.csv') # Change the name of the file to what you have it named here
    logTuples = lines.filter( lambda line: len(line) > 0) \
                     .zipWithIndex() \
                     .filter( lambda x: x[1] > 0) \
                     .map(lambda x: x[0]) \
                     .map( lambda line: line.split(',')) \
                     .map( lambda arr: ((arr[11][:4], arr[24]), float(arr[16])))
    stateRanks = logTuples.mapValues( lambda v: (v, 1)) \
                          .reduceByKey( lambda a,b: (a[0]+b[0], a[1]+b[1])) \
                          .mapValues( lambda v: v[0]/v[1]) \
                          .sortBy(lambda x: (x[0][0], x[1]))
    for year_state, value in stateRanks.take(100):
        year, state = year_state
        print(year, state, value)
    sc.stop()
except Exception as err:
    print(err)
    sc.stop()

In [31]:
!!time python ./P3/spark_p3.py > ./P3/p3.result

['21/12/22 17:07:58 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable',
 "Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties",
 'Setting default log level to "WARN".',
 'To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).',
 '',
 '[Stage 0:>                                                          (0 + 4) / 4]',
 '                                                                                ',
 '',
 '[Stage 1:>                                                          (0 + 4) / 4]',
 '[Stage 1:==============>                                            (1 + 3) / 4]',
 '                                                                                ',
 'real\t0m9.334s',
 'user\t0m0.750s',
 'sys\t0m0.830s']

In [32]:
!cat ./P3/p3.result

1990 West Virginia 0.0
1990 Oklahoma 0.0
1990 Wisconsin 0.0
1990 Virgin Islands 0.0
1990 Hawaii 0.00019703703703703704
1990 Nevada 0.00042080000000000004
1990 Alaska 0.00044208333333333334
1990 South Dakota 0.0005705
1990 Washington 0.0005974999999999999
1990 Wyoming 0.0006045454545454545
1990 Utah 0.0007970588235294118
1990 New Mexico 0.0008222222222222222
1990 Oregon 0.0008596296296296297
1990 Arizona 0.0008620134228187919
1990 Maine 0.0009789285714285713
1990 Colorado 0.0021623741007194244
1990 Mississippi 0.0026666666666666666
1990 Missouri 0.0056
1990 Michigan 0.006559896373056996
1990 Connecticut 0.0081
1990 Georgia 0.008366666666666666
1990 Puerto Rico 0.01005
1990 North Carolina 0.0143
1990 Alabama 0.024325
1990 Iowa 0.0332
1990 Pennsylvania 0.1059625
1990 Ohio 0.135716
1990 Illinois 0.14575701219512197
1990 New Jersey 0.2933352941176471
1990 Minnesota 0.306488
1990 California 0.41153099836333895
1990 South Carolina 0.5598140350877193
1990 District Of Columbia 0.826150819672131

### P4

In [ ]:
# %load ./P4/spark_p4.py
from typing import Sequence
from pyspark.sql import *
from pyspark.sql.types import *
from pyspark.sql.functions import *
from math import sqrt as math_sqrt
from math import pow as power

spark = SparkSession.builder.master('local[*]').appName('words').getOrCreate()
sc = spark.sparkContext

try:
    # files
    lines_states = sc.textFile('../data/usa_states.csv')
    lines = sc.textFile('../data/epa_hap_daily_summary-small.csv')

    # file mapping
    logRows_states = lines_states.filter( lambda line : len(line) > 0)    \
                    .zipWithIndex() \
                    .filter( lambda x: x[1] > 0) \
                    .map(lambda x: x[0]) \
                    .map( lambda line: line.split(',')) \
                    .map( lambda arr : (arr[1], float(arr[2]), float(arr[3]), \
                                        float(arr[4]), float(arr[5])))
    logRows = lines.filter( lambda line: len(line) > 0) \
                     .zipWithIndex() \
                     .filter( lambda x: x[1] > 0) \
                     .map(lambda x: x[0]) \
                     .map( lambda line: line.split(',')) \
                     .map( lambda arr: (arr[24], (float(arr[5]), float(arr[6]))))

    logRows = logRows.distinct()

    final = logRows_states.map(lambda arr: (arr[0], (((arr[1]+arr[2])/2), ((arr[3]+arr[4])/2))))\
                    .join(logRows) \
                    .map(lambda arr: (arr[0], math_sqrt(power( (arr[1][1][0]-arr[1][0][0])*111,2) +power( (arr[1][1][1]-arr[1][0][1])*111,2)))) \
                    .mapValues(lambda v: (v, 1)) \
                    .reduceByKey(lambda a,b: (a[0]+b[0], a[1]+b[1])) \
                    .mapValues(lambda v: v[0]/v[1]) \

    
    for k,v in final.take(20):
        print(k,v)

    sc.stop()
    
except Exception as err:
    print(err)
    sc.stop()


In [13]:
!!time python ./P4/spark_p4.py > ./P4/p4.result

['21/12/22 16:54:10 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable',
 "Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties",
 'Setting default log level to "WARN".',
 'To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).',
 '',
 '[Stage 0:>                                                          (0 + 2) / 2]',
 '                                                                                ',
 '',
 '[Stage 1:>                                                          (0 + 4) / 4]',
 '[Stage 1:==============>                                            (1 + 3) / 4]',
 '                                                                                ',
 '',
 '[Stage 2:>                                                          (0 + 4) / 4]',
 '[Stage 2:=============================>                             (2 + 2) / 4]',
 '                   

In [14]:
!cat ./P4/p4.result

Arizona 178.87574055612174
Delaware 51.57977048023351
Florida 336.54491465708315
North Carolina 179.09404815356976
New Hampshire 115.6220509234631
New Jersey 80.74367303780123
New Mexico 183.18912128523766
Oregon 268.8538079232643
South Carolina 131.4918866976489
Vermont 504.0632354850809
West Virginia 144.49363881493753
Puerto Rico 32.73151627576544
Alaska 603.6996422410685
California 328.2263813155367
Idaho 289.635073275631
Kentucky 219.95151680752102
Mississippi 174.2386099835142
Nevada 326.28118071973915
New York 283.7273398637172
Oklahoma 236.88257437298154


### P5

In [ ]:
# %load ./P5/spark_p5.py
#!/usr/bin/env python
import pyspark

totalExecTime = 0
sc = pyspark.SparkContext('local[*]')
try :
    lines = sc.textFile('../data/epa_hap_daily_summary-small.csv')
    usa_states_lines = sc.textFile('../data/usa_states.csv')

    logTuples = lines.filter( lambda line: len(line) > 0) \
                     .zipWithIndex() \
                     .filter( lambda x: x[1] > 0) \
                     .map(lambda x: x[0]) \
                     .map( lambda line: line.split(',')) \
                     .map( lambda arr: (arr[24], (float(arr[5]),float(arr[6])) )) \
                     .distinct() #ignore repeated entries with the same state, lat and lon
                        
                     #.map( lambda arr: ((arr[11][:4], arr[24]), float(arr[16])))
    
    statesTuples = usa_states_lines.filter( lambda line: len(line) > 0) \
                     .zipWithIndex() \
                     .filter( lambda x: x[1] > 0) \
                     .map(lambda x: x[0]) \
                     .map( lambda line: line.split(',')) \
                     .map( lambda arr: (arr[1], [float(arr[2]),float(arr[3]),float(arr[4]),float(arr[5])] ))\
                     .mapValues(lambda x: ((x[0]+x[1])/2,(x[2]+x[3])/2))  #calculate midlat and midlon
    
    tuplesJoined = logTuples.join(statesTuples).mapValues(lambda x: [x[0][0],x[1][0],x[0][1],x[1][1]]) 
    
    stateCoords1stQuad = tuplesJoined.filter(lambda x: float(x[1][0]) < float(x[1][1]) and float(x[1][2]) < float(x[1][3]))                                 
    stateCoords2ndQuad = tuplesJoined.filter(lambda x: float(x[1][0]) < float(x[1][1]) and float(x[1][2]) > float(x[1][3]))
    stateCoords3rdQuad = tuplesJoined.filter(lambda x: float(x[1][0]) > float(x[1][1]) and float(x[1][2]) < float(x[1][3]))
    stateCoords4thQuad = tuplesJoined.filter(lambda x: float(x[1][0]) > float(x[1][1]) and float(x[1][2]) > float(x[1][3]))
    
    stateMonitors1stQuad = stateCoords1stQuad.mapValues(lambda x: (1)) \
                                             .reduceByKey(lambda a,b: a+b)\
                                             .mapValues(lambda x: ('NW',x+0))
                                             
    
    stateMonitors2ndQuad = stateCoords2ndQuad.mapValues(lambda x: (1)) \
                                             .reduceByKey(lambda a,b: a+b)\
                                             .mapValues(lambda x: ('NE',x+0))
    
    stateMonitors3rdQuad = stateCoords3rdQuad.mapValues(lambda x: (1)) \
                                             .reduceByKey(lambda a,b: a+b)\
                                             .mapValues(lambda x: ('SW',x+0))
    
    stateMonitors4thQuad = stateCoords4thQuad.mapValues(lambda x: (1)) \
                                             .reduceByKey(lambda a,b: a+b)\
                                             .mapValues(lambda x: ('SE',x+0))
    
    #join won't do because some states dont #have any monitors in certain quadrants #like Florida (NW)
    #fullOuterJoin works, setting the quadrant's value as None if there are no monitors in that same quadrant
    
    finalStateMonitorsPerQuad = stateMonitors1stQuad.fullOuterJoin(stateMonitors2ndQuad)\
                                                    .fullOuterJoin(stateMonitors3rdQuad)\
                                                    .fullOuterJoin(stateMonitors4thQuad)
    
    for (key,value) in finalStateMonitorsPerQuad.collect():
        print(key,value)
    
    sc.stop()
except Exception as e:
    print(e)
    sc.stop()

In [16]:
!!time python ./P5/spark_p5.py > ./P5/p5.result

['21/12/22 16:54:24 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable',
 "Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties",
 'Setting default log level to "WARN".',
 'To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).',
 '',
 '[Stage 0:>                                                          (0 + 4) / 4]',
 '                                                                                ',
 '',
 '[Stage 2:>                                                          (0 + 4) / 4]',
 '[Stage 4:===================>                                       (2 + 4) / 6]',
 '                                                                                ',
 'real\t0m11.302s',
 'user\t0m0.943s',
 'sys\t0m0.824s']

In [17]:
!cat ./P5/p5.result

New Mexico (((('NW', 3), ('NE', 2)), ('SW', 12)), ('SE', 1))
Arizona (((('NW', 10), ('NE', 16)), ('SW', 10)), ('SE', 2))
New Jersey (((('NW', 4), ('NE', 1)), ('SW', 1)), ('SE', 18))
Vermont (((('NW', 5), ('NE', 1)), ('SW', 16)), ('SE', 1))
South Carolina (((('NW', 3), ('NE', 18)), ('SW', 33)), ('SE', 10))
West Virginia (((('NW', 5), None), ('SW', 3)), ('SE', 2))
Oregon (((('NW', 13), ('NE', 1)), ('SW', 15)), ('SE', 3))
New Hampshire (((('NW', 4), ('NE', 11)), None), ('SE', 2))
Delaware (((('NW', 2), None), ('SW', 4)), None)
Puerto Rico (((('NW', 1), None), ('SW', 1)), ('SE', 4))
Florida (((None, ('NE', 23)), ('SW', 5)), ('SE', 27))
North Carolina (((None, ('NE', 5)), ('SW', 19)), ('SE', 26))
California (((('NW', 16), ('NE', 68)), ('SW', 84)), ('SE', 2))
Wisconsin (((('NW', 1), ('NE', 21)), ('SW', 2)), ('SE', 2))
New York (((('NW', 3), ('NE', 43)), ('SW', 16)), ('SE', 5))
Kentucky (((('NW', 16), ('NE', 2)), ('SW', 3)), ('SE', 13))
Mississippi (((('NW', 5), ('NE', 12)), None), ('SE', 4))